In [ ]:
import numpy as np
import time
import pandas

In [ ]:
def compute_mae(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [ ]:
def laplacian(n, tau):
    ell = np.zeros(n)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox(z, w, lmbda, denominator):
    T = z.shape[0]
    temp = np.fft.fft(lmbda * z - w) / denominator
    temp1 = 1 - T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)


In [ ]:
ef LCR(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    denominator = lmbda + gamma * np.fft.fft(laplacian(T, tau)) ** 2
    T=y_true.shape[0]
    del y_true, y
    show_iter = 10
    for it in range(maxiter):
        x = prox(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
    print(f'lambda: {lmbda/T}, gamma: {gamma/lmbda}, tau: {tau}')
    print(f'MAE Accuracy: {compute_mae(y_test, x[pos_test])}')
    print(f'RMSE Accuracy: {compute_rmse(y_test, x[pos_test])}')
    print(round(compute_mae(y_test, x[pos_test]),5))
    print(round(compute_rmse(y_test, x[pos_test]),5)
    print()
    return x

In [ ]:
batch_size=20
df = pandas.read_csv('dataset1.csv',nrows=batch_size)
missing_rate = 0.20
print('Missing rate = {}'.format(missing_rate))
dense_mat = df.to_numpy()
dense_vec = dense_mat[:,0]  
T = dense_vec.shape[0]
np.random.seed(1)
sparse_vec = dense_vec * np.round(np.random.rand(T) + 0.5 - missing_rate)
start_time = time.time()
lmbda = 5e-1 * T
gamma = 3 * lmbda
tau = 2
maxiter = 100
x = LCR(dense_vec, sparse_vec, lmbda, gamma, tau, maxiter)
lmbda = 5e-2 * T
gamma = 1 * lmbda
tau = 2
maxiter = 100
x = LCR(dense_vec, sparse_vec, lmbda, gamma, tau, maxiter)
print("--- %s seconds ---" % (time.time() - start_time))